### Ejercicio 4

Para realizar nuestra simulacion de Simpy, definimos nuestro escenario Banco, al cual le asignamos un recurso de capacidad 1, es decir, que sólo una persona a la vez puede estar utilizando nuestro recurso ATM a la vez.

Para la generacion de la llegada de personas, aplicamos una distribución exponencial, y por cada llegada, creamos una nueva Persona, que intenta utilizar el recurso.

El tiempo de utilización del recurso se basa en una distribución uniforme, con probabilidades dependiendo del tipo de persona que llega. Además, consideramos un grado de tolerancia, que también se basa en una distribución uniforme.



In [6]:
import simpy
import numpy as np
import random

#Definimos banco como enviroment. 
def banco(env):
    #Creamos un recurso atm, con capacidad 1
    atm = simpy.Resource(env,capacity=1)

    print("Esperando que abra el banco...")
    yield env.timeout(10*3600)
    print("Abrimos!")

    i = 0;
    while True:  
       #Calculo la llegada de las personas
        t = random.expovariate(frecuenciaArribo(env))
        yield env.timeout(t)
        #Creamos la persona que llego
        persona = Persona() 
        i+=1;
        persona.nombre = str(i);
        print(getTimeFromSeconds(env.now), "Llega al cajero ", persona.tipo + "-" + persona.nombre)
        env.process(cajero(env, atm, persona))
        
#Se encarga de manejar el pedido de nuestra atm    
def cajero(env,atm,persona):
        with atm.request() as req:
            yield req
            tiempoIngreso = env.now
            yield env.timeout(persona.tiempoCajero*60)
            print(getTimeFromSeconds(env.now), " Sale del cajero: ", persona.tipo + "-" + persona.nombre)
            print("---Tiempo de uso --: ", getMinutesFromSeconds(env.now - tiempoIngreso))
    
            
def getTimeFromSeconds(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    return "%d:%02d:%02d" % (h, m, s)

def getMinutesFromSeconds(seconds):
    m, s = divmod(seconds, 60)
    return "%02d:%02d" % (m, s)

#Se encarga de obtener la frecuencia con la cual llega la persona
def frecuenciaArribo(env):
    if (env.now <= 12*3600): #10 a 12
        return 1.0/260.0
    if (env.now <= 15*3600):# 12 a 15
        return 1.0/120.0
    return 1.0/360.0 # 15 a 19


class Persona:
    tipo = ""
    tiempoCajero = 1
    nombre = ""
    
    def __init__(self, tipo,tiempoCajero):
        self.tipo = tipo
        self.tiempoCajero = tiempoCajero
        
    #Definimos el tipo de persona que es, y con eso seteamos un tiempo de permanencia en el cajero.    
    def __init__(self): 
        val = random.uniform(0,1);
        if (val <= 0.1):
            #print("--Persona tipo 1--")
            self.tipo = "Tipo 1" #Tolerancia +-3
            if (random.uniform(0,1) >= .5):
                self.tiempoCajero = 4.0 + (np.random.uniform(0,3));
            else:
                self.tiempoCajero = 4.0 + (np.random.uniform(0,3))*-1;
        elif (val <= .8):
            #print("--Persona tipo 2--")
            self.tipo = "Tipo 2" #Tolerancia +-1
            if (random.uniform(0,1) >= .5):
                self.tiempoCajero = 2.0 + np.random.uniform(0,1); 
            else:    
                self.tiempoCajero = 2.0 + np.random.uniform(0,1)*-1; 

        #print("--Persona tipo 3--")
        else:
            self.tipo = "Tipo 3" #Tolerancia 3+-2
            if (random.uniform(0,1) >= .5):
                self.tiempoCajero = 3.0 + np.random.uniform(0,2); 
            else:    
                self.tiempoCajero = 3.0 + np.random.uniform(0,2)*-1;
                

#Seteamos el ambiente y lo corremos hasta que sean las 19 hs.
env = simpy.Environment()
env.process(banco(env))
env.run(until=19*3600)
print("Banco Cerrado !")

Esperando que abra el banco...
Abrimos!
('10:03:25', 'Llega al cajero ', 'Tipo 2-1')
('10:03:35', 'Llega al cajero ', 'Tipo 2-2')
('10:03:56', 'Llega al cajero ', 'Tipo 2-3')
('10:05:00', ' Sale del cajero: ', 'Tipo 2-1')
('---Tiempo de uso --: ', '01:35')
('10:07:47', ' Sale del cajero: ', 'Tipo 2-2')
('---Tiempo de uso --: ', '02:47')
('10:09:07', 'Llega al cajero ', 'Tipo 1-4')
('10:10:07', ' Sale del cajero: ', 'Tipo 2-3')
('---Tiempo de uso --: ', '02:19')
('10:14:35', 'Llega al cajero ', 'Tipo 2-5')
('10:14:40', 'Llega al cajero ', 'Tipo 2-6')
('10:15:53', ' Sale del cajero: ', 'Tipo 1-4')
('---Tiempo de uso --: ', '05:46')
('10:18:47', ' Sale del cajero: ', 'Tipo 2-5')
('---Tiempo de uso --: ', '02:53')
('10:19:28', 'Llega al cajero ', 'Tipo 2-7')
('10:21:27', ' Sale del cajero: ', 'Tipo 2-6')
('---Tiempo de uso --: ', '02:40')
('10:23:35', ' Sale del cajero: ', 'Tipo 2-7')
('---Tiempo de uso --: ', '02:07')
('10:47:43', 'Llega al cajero ', 'Tipo 2-8')
('10:50:17', ' Sale del ca